## Tests on deprecated 1D code

In [ ]:
# model = gpr.sparseGPR(kernel,num_datapoints,num_derivpoints,referencegrid,sparse_method="ppa",noise_var=noise**2)
# model.fit(X_data,Y_data)
# (means_ppa, stds_ppa) = model.predict(grid,True)

# model = gpr.sparseGPR(kernel,num_datapoints,num_derivpoints,referencegrid,sparse_method="sor",noise_var=noise**2)
# model.fit(X_data,Y_data)
# (means_sor, stds_sor) = model.predict(grid,True)

# model = gpr.sparseGPR(gpr.RBF_1D(1.5),num_datapoints,num_derivpoints,referencegrid,sparse_method="ny",noise_var=noise**2)
# model.fit(X_data,Y_data)
# (means_ny, stds_ny) = model.predict(grid,True)

In [ ]:
# plt.plot(grid,means_ppa,label="ppa")
# plt.fill_between(grid,means_ppa-stds_ppa,means_ppa+stds_ppa,alpha=0.5)
# plt.plot(grid,means_sor,label="sor")
# plt.fill_between(grid,means_sor-stds_sor,means_sor+stds_sor,alpha=0.5)
# plt.plot(grid,means_ny,label="ny")
# plt.fill_between(grid,means_ny-stds_ny,means_ny+stds_ny,alpha=0.5)
# plt.plot(grid,func(grid),color="grey",ls="--",label="sin(x)")
# plt.scatter(datagrid,data_values,color="r",label="function values")
# plt.grid()
# plt.legend()